In [ ]:
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import Normalize

from sklearn.datasets import fetch_openml
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.exceptions import ConvergenceWarning

import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

from fdshapley import FederatedShapley

In [ ]:
# Load data from https://www.openml.org/d/554
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)

In [ ]:
def split(X, y, n_participants, frac_test=0.2):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 
    xs_train, ys_train = np.split(X_train, n_participants), np.split(y_train, n_participants)
    return  xs_train, ys_train, X_test, y_test

In [ ]:
xs_train, ys_train, X_test, y_test = split(X[:10_000], y[:10_000], 10, 0.2)

In [ ]:
params = {
    'max_iter': 1, 
    'warm_start': True,
    'fit_intercept':False
}

In [ ]:
data_train = [[x,y] for x,y in zip(xs_train, ys_train)]
data_test = [X_test, y_test]

for i in [0,1,2,3, 4]:
        data_train[i][1] = np.random.randint(0,10, len(data_train[i][1]))


In [ ]:
fed = FederatedShapley(data_train, data_test)
res, first = fed.federatedSVEstimation(0.2, 50)

In [ ]:
fed.u(fed.w)

In [ ]:
cmap = cm.hot
norm = Normalize(vmin=0, vmax=10)

colors =  [cmap(norm(x)) for x in first]
plt.bar(range(10), res, color=colors)